In [1]:
from supervised.automl import AutoML

import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns

from helpers import *

In [2]:
# algos = ['CatBoost']



training_filters = [
    {
        'feature': 'month',
        'operator': 'between',
        'value': [3, 9]
    }
]

validation_filters = [
    {
        'feature': 'date_forecast',
        'operator': 'between',
        'value': ['2020-05-01', '2020-07-01']
    }
]

test_filters = [
    {
        'feature': 'date_forecast',
        'operator': 'between',
        'value': ['2021-05-01', '2021-07-01']
    }
]


agg_strats = ['min','max','mean','std']

m_wrap = DataWrapper(
    impute_strategy = 'fbfill',
    aggregation_strats = agg_strats,
    y_scalers = [Y_Scaler_MaxAbs_per_building()],
    transform_pred_strategy = ['clip','sun_el_thr'],
    training_filters=training_filters,
    validation_filters=validation_filters,
    test_filters=test_filters,
    bagging_filters=False, # these are applied additionally to the other filters on val, test, sub
    )

# X_train_agg, y_train_agg = m_wrap.get_train(aggregated=True,drop_val=False,drop_test=False,y_scaled=True)
X_sub_agg = m_wrap.get_sub(aggregated=True)

# train_idx = X_train_agg[~X_train_agg['date_forecast'].between('2020-05-01', '2020-07-01')].index
# val_idx = X_train_agg[X_train_agg['date_forecast'].between('2020-05-01', '2020-07-01')].index

# X_train_agg = X_train_agg.select_dtypes(exclude=['datetime','timedelta','period[M]'])

# cv = [(train_idx, val_idx)]

mod = AutoML(
    results_path='AutoML_Results_ALL/',
    # mode='Compete',
    # total_time_limit=60*60*8,
    # eval_metric='mae',
    # algorithms=algos,
    # train_ensemble=True,
    # stack_models=True,
    # verbose=2,
    # random_state=42,
    # validation_strategy={"validation_type": "custom"}
)

# mod.fit(X_train_agg, y_train_agg, cv=cv)

In [3]:
pred = pd.Series(mod.predict(X_sub_agg))
pred

/opt/homebrew/Caskroom/miniconda/base/envs/ML/lib/python3.11/site-packages/sklearn/base.py:347: InconsistentVersionWarning: Trying to unpickle estimator MiniBatchKMeans from version 1.3.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
/opt/homebrew/Caskroom/miniconda/base/envs/ML/lib/python3.11/site-packages/sklearn/base.py:347: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.3.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations


0      -0.000172
1      -0.000171
2       0.000095
3       0.010848
4       0.061846
          ...   
2155    0.044339
2156    0.014237
2157    0.003404
2158   -0.000670
2159   -0.000900
Length: 2160, dtype: float64

In [4]:
prediction = m_wrap.transform_prediction(pred,X_sub_agg)
prediction

0         0.000000
1         0.000000
2         0.544771
3        62.197813
4       354.589862
           ...    
2155     44.320943
2156     14.230972
2157      3.402735
2158      0.000000
2159      0.000000
Length: 2160, dtype: float64

In [5]:
prediction = m_wrap.transform_prediction(pred,X_sub_agg,0)
prediction

0         0.000000
1         0.000000
2         0.000000
3        62.197813
4       354.589862
           ...    
2155     44.320943
2156     14.230972
2157      3.402735
2158      0.000000
2159      0.000000
Length: 2160, dtype: float64

In [10]:
prediction.to_csv('Auto_ALL.csv',index=True)

In [7]:
l = [0,1,2,3,4]
sns.lineplot(l)

/opt/homebrew/Caskroom/miniconda/base/envs/ML/lib/python3.11/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
/opt/homebrew/Caskroom/miniconda/base/envs/ML/lib/python3.11/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
/opt/homebrew/Caskroom/miniconda/base/envs/ML/lib/python3.11/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
/opt/homebrew/Caskroom/miniconda/base/envs/ML/lib/python3.11/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.


<Axes: >

In [8]:
plt.plot(l);
plt.show()

/var/folders/s_/tyyjfphn3wl93jz8bhkrh3x40000gn/T/ipykernel_51325/2941895085.py:2: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown


In [9]:
sns.lineplot(prediction)
plt.show()

/opt/homebrew/Caskroom/miniconda/base/envs/ML/lib/python3.11/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
/opt/homebrew/Caskroom/miniconda/base/envs/ML/lib/python3.11/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
/opt/homebrew/Caskroom/miniconda/base/envs/ML/lib/python3.11/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
/opt/homebrew/Caskroom/miniconda/base/envs/ML/lib/python3.11/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
/var/folders/s_/tyyjfphn3wl93jz8